In [1]:
import pandas as pd
from tqdm import tqdm
import csv

In [2]:
# Сгенерируем колонки
def generate_columns(n, name):
    columns = [name+str(i) for i in range(1, n+1)]
    return columns

In [3]:
# Из общего файла n-грамм выкидываем инфу про хостов
# Остаются только сами последовательности слов*
# *не самих лексем, а их частей речи
def without_host(n, my_columns, file_dir):
    df_all = pd.read_csv(file_dir + f"/all_start_finish/all_count_{n}_grams.csv", sep=",")
    df_wth_host = df_all[my_columns + ["total_entries"]]
    return df_wth_host

In [4]:
# Группируем последовательности ЧР 
# Добавляем информацию про число вхождений каждого типа последовательности
# и число различных видов подчинения внутри каждого типа
# записываем в файл
def order_entries_and_reltypes(n, my_columns, file_dir):
    df_wth_host = without_host(n, my_columns, file_dir=file_dir)
    df_grouped = df_wth_host.groupby(my_columns, as_index=False).agg({"total_entries": sum})
    df_grouped["number_of_rel_types"] = df_wth_host.groupby(my_columns, as_index=False)\
                                        .agg({"total_entries": "count"})["total_entries"]
    df_grouped = df_grouped.sort_values(by=["total_entries", "number_of_rel_types"], ascending=[False, False])
    df_grouped = df_grouped.rename(columns={'total_entries': 'total_entries_of_construction'})
    df_grouped.to_csv(file_dir + f"/all_start_finish/all_order_entries_and_reltypes_{n}.csv", index=None)

In [5]:
for i in range(3,7):
    order_entries_and_reltypes(i, generate_columns(i, "POS"), "./outcome files/dev_train_test")

In [6]:
for i in range(3,7):
    order_entries_and_reltypes(i, generate_columns(i, "POS"), "./outcome files/dev_train")

**Мини-выводы (для 4-грамм):**
- Полученные типы можно условно разделить на 3 группы:
    1. Мало вхождений типа (таких больше всего, но нас не интересуют, т.к. по ним не можем ничего утверждать)
    2. Много вхождений типа и мало подтипов (супер крутые! наша цель)
    3. Много вхождений, но много типов подчинения (среди них тоже есть хорошие!)
- Типов.очень.много (18775), даже только того, что нас интересует (п.2 и 3), всё равно очень много
- Типы подчинения могут быть распределены по-разному! (Условно, мб 2 типа подчинения, но они пополам, это плохо. А может быть 4 типа, но 1 в 99% случаях, а остальные по одному разу встречаются -- и это очень хорошо!)

Надо посчитать распределение типов подчинения внутри типа n-граммы

## Доли типов подчинения внутри типов n-грамм (по одной связи)

In [7]:
# (код (почти) здорового человека)
# Из всех n-грамм оставляем только те, что встречаются чаще порога lim
# Добавляем колонку с номерами конструкций
# Соединяем два df по ключам-частям речи
# Дробаем лишнее и переставляем покрасивее
def df_all_for_ratio(n, lim, file_dir):
    df_to_stay = pd.read_csv(file_dir + f"/all_start_finish/all_order_entries_and_reltypes_{n}.csv", sep=",")
    df_to_stay = df_to_stay[df_to_stay["total_entries_of_construction"] > lim]
    df_to_stay["#construction"] = df_to_stay.index
    
    df_all = pd.read_csv(file_dir + f"/all_start_finish/all_count_{n}_grams.csv", sep=",")
    df_all = pd.merge(df_all, df_to_stay, on=generate_columns(n, "POS"))

    df_all = df_all.drop(["number_of_rel_types"], axis=1)
    df_all = df_all[["#construction"] + list(df_all.columns.values)[:-1]]
    df_all = df_all.sort_values(generate_columns(n, "POS")).reset_index(drop=True)
    
    return df_all

In [8]:
# Словарик для группировки по конструкциям
def create_agg_dict(n):
    agg_dict = {"total_entries": 'sum', "total_entries_of_construction": 'first', 
                "#construction": 'first'}
    for col in generate_columns(n, "POS"):
        agg_dict[col] = 'first' 
    return agg_dict

In [9]:
# Заполнитель нулями (0 = на это не смотрим)
def place_zero(i, n, constr_ratio):
    for i_n in  range (1, n + 1):
        if i_n != i:
            constr_ratio[f"#host{i_n}"] = 0
            constr_ratio[f"POShost{i_n}"] = 0

In [10]:
# Вытаксиваем отношения для i-го слова в конструкции
def ratio_from_construction(i, n, constr, agg_dict): 
    constr_ratio = constr.groupby([f"#host{i}", f"POShost{i}"], as_index=False).agg(agg_dict)

    constr_ratio["ratio"] = round(constr_ratio["total_entries"] / 
                                  constr_ratio["total_entries_of_construction"], 3)
    constr_ratio = constr_ratio.sort_values(by="ratio", ascending=False).reset_index(drop=True)
    place_zero(i, n, constr_ratio)
    constr_ratio = constr_ratio.drop(["total_entries_of_construction"], axis=1)
    return constr_ratio

In [11]:
# Создаем csv файл для дальнейшей записи по кусочкам
def init_csv(n, file_dir):
    with open(file_dir + f"/all_start_finish/all_with_ratios_{n}.csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',')
        csvwriter.writerow(right_col_order(n))

In [12]:
# Правильный порядок для записи
def right_col_order(n):
    right_order = ["#construction"]
    for i in range(n):
        right_order.extend([generate_columns(n, "POS")[i], 
                            generate_columns(n, "#host")[i], 
                            generate_columns(n, "POShost")[i]])
    right_order.extend(["ratio", "total_entries"])
    return right_order

In [13]:
# Записываем в csv кусочек конструкций
def get_result(n, ratio_from_construction, file_dir):
    result = ratio_from_construction[right_col_order(n)].reset_index(drop=True)
    with open(file_dir + f"/all_start_finish/all_with_ratios_{n}.csv", 'a') as csvadd:
        result.to_csv(csvadd, header=False, index=None)

In [14]:
# lim -- минимальное число вхождений конструкции
# constrs_num -- список номеров конструкций

# Общая функция (! всё равно долго, и чем больше n, тем дольше)
def ratios_of_reltypes(n, lim, file_dir):
    init_csv(n, file_dir=file_dir)

    df_all = df_all_for_ratio(n, lim, file_dir=file_dir)
    constrs_num = pd.unique(df_all["#construction"]).tolist()
    agg_dict = create_agg_dict(n)

    for num in tqdm(constrs_num):
        constr = df_all[df_all["#construction"] == num].reset_index()
        for i in range(1, n + 1):
            get_result(n, ratio_from_construction(i, n, constr, agg_dict), file_dir=file_dir)

In [15]:
ratios_of_reltypes(3, 0, "./outcome files/dev_train_test")
ratios_of_reltypes(3, 0, "./outcome files/dev_train")

100%|██████████████████████████████████████████████████████████████████████████████| 2844/2844 [03:59<00:00, 11.86it/s]


In [16]:
# ratios_of_reltypes(4, 0, "./outcome files/dev_train_test")
# ratios_of_reltypes(4, 0, "./outcome files/dev_train")

In [17]:
# ratios_of_reltypes(5, 0, "./outcome files/dev_train_test")
# ratios_of_reltypes(5, 0, "./outcome files/dev_train")

In [18]:
# ratios_of_reltypes(6, 0, "./outcome files/dev_train_test")
# ratios_of_reltypes(6, 0, "./outcome files/dev_train")

## Доли типов подчинения внутри типов n-грамм (все связи)

In [16]:
def ratios_of_reltypes_all(n, lim, file_dir):
    df_all = df_all_for_ratio(n, lim, file_dir=file_dir)
    df_all["ratio"] = round(df_all["total_entries"] / df_all["total_entries_of_construction"], 3)
    df_all = df_all.sort_values(generate_columns(n, "POS")).reset_index(drop=True)
    df_all.to_csv(file_dir + f"/all_start_finish/all_count_{n}_grams.csv", index=None)

In [17]:
for i in range(3, 7):
    ratios_of_reltypes_all(i, 0, "./outcome files/dev_train_test")

In [18]:
for i in range(3, 7):
    ratios_of_reltypes_all(i, 0, "./outcome files/dev_train")

In [34]:
df3 = pd.read_csv(f"./outcome files/dev_train_test/all_start_finish/all_count_3_grams.csv", sep=",", low_memory=False)

In [35]:
df3 = df3[(df3["ratio"] >= 0.97) & 
          ((df3["#host1"] != "_") |
           (df3["#host2"] != "_") |
           (df3["#host3"] != "_"))].sort_values(generate_columns(3, "POS"))

In [37]:
def get_recall(df, n):
    map_dict = {str(i): 1 for i in range(1, n + 1)}
    map_dict["_"] = 0
    df["sum"] = 0
    for i in range(1, n + 1):
        df[f"num_host{str(i)}"] = df[f"#host{i}"].map(map_dict)
        df["sum"] += df[f"num_host{str(i)}"]
    df["num_links"] = df["total_entries"] * df["sum"]
    return df["num_links"].sum()

In [38]:
get_recall(df3, 3)

18893

In [39]:
df4 = pd.read_csv(f"./outcome files/dev_train_test/all_start_finish/all_count_4_grams.csv", sep=",", low_memory=False)
df4 = df4[(df3["ratio"] >= 0.97) & 
          ((df4["#host1"] != "_") |
           (df4["#host2"] != "_") |
           (df4["#host3"] != "_") |
           (df4["#host4"] != "_"))].sort_values(generate_columns(4, "POS"))

In [41]:
print(get_recall(df3, 3))
print(get_recall(df4, 4))

18893
20039


## Посмотреть только на крутые единички

In [26]:
def cool_ones(n):
    df_from_ratios = pd.read_csv(f"./outcome files/all_start_finish/all_with_ratios_{n}.csv", sep=",", low_memory=False)
#     col_hosts = generate_columns(n, "#host")
#     contitions = [(df_from_ratios[col] != "_") for col in col_hosts]
#     conditions.append((df_from_ratios["ratio"] == 1.0))
#     # Как любой список условий объединить через &?
    return df_from_ratios[((df_from_ratios["ratio"] == 1.0) & 
                          (df_from_ratios["#host1"] != "_") &
                          (df_from_ratios["#host2"] != "_") &
                          (df_from_ratios["#host3"] != "_") &
                          (df_from_ratios["#host4"] != "_"))]

In [40]:
cool_4.sort_values(by="total_entries", ascending=False).head(20)

,#construction,POS1,#host1,POShost1,POS2,#host2,POShost2,POS3,#host3,POShost3,POS4,#host4,POShost4,ratio,total_entries
53036,91,NOUN,0,0,DET,3,NOUN,NOUN,0,0,PUNCT,0,0,1.0,1246
3577,256,ADJ,0,0,NOUN,0,0,DET,4,NOUN,NOUN,0,0,1.0,601
107799,384,PUNCT,0,0,SCONJ,0,0,DET,4,NOUN,NOUN,0,0,1.0,442
26848,453,ADV,0,0,VERB,0,0,DET,4,NOUN,NOUN,0,0,1.0,380
54411,455,NOUN,0,0,NOUN,0,0,PART,4,VERB,VERB,0,0,1.0,376
44181,478,DET,0,0,NOUN,0,0,PART,4,VERB,VERB,0,0,1.0,365
86264,504,PRON,0,0,VERB,0,0,DET,4,NOUN,NOUN,0,0,1.0,347
18,505,ADJ,0,0,ADJ,0,0,ADJ,4,NOUN,NOUN,0,0,1.0,346
124070,535,VERB,0,0,NOUN,0,0,DET,4,NOUN,NOUN,0,0,1.0,333
301,616,ADJ,0,0,ADJ,3,NOUN,NOUN,0,0,PROPN,0,0,1.0,302
